In [11]:
%run ../../Utils/yp_utils.py

# Initial setup

In [12]:
paper_pmid = 29889214
paper_name = 'wong_khalil_2018' 

In [13]:
datasets = pd.read_csv('extras/YeastPhenome_' + str(paper_pmid) + '_datasets_list.txt', sep='\t', header=None, names=['dataset_id', 'name'])

In [14]:
datasets.set_index('dataset_id', inplace=True)

In [25]:
datasets.shape

(48, 1)

# Load & process the data

In [15]:
original_data = pd.read_csv('raw_data/YKO_HS_fitness.csv', sep=',')

In [16]:
print('Original data dimensions: %d x %d' % (original_data.shape))

Original data dimensions: 5153 x 50


In [17]:
original_data.head()

,Unnamed: 0,Temperature Magnitude,30,33,33.1,33.2,36,36.1,36.2,36.3,...,36.10,36.11,39.8,39.9,39.10,39.11,42.8,42.9,42.10,42.11
0,NaN,Temperature Period,STEP,2,6,48,STEP,2,6,48,...,6,48,STEP,2,6,48,STEP,2,6,48
1,NaN,Timepoint,Day 2,Day 2,Day 2,Day 2,Day 2,Day 2,Day 2,Day 2,...,Day 6,Day 6,Day 6,Day 6,Day 6,Day 6,Day 6,Day 6,Day 6,Day 6
2,NaN,Generations,21.21080202,31.299567,25.27666953,23.0038688,23.61190272,25.99524616,25.75489888,23.51884228,...,79.22577137,72.78772317,44.76736069,86.41605256,66.74935062,64.47573334,7.292367769,63.98352944,74.14534641,34.10083037
3,Gene,Sequence,'30C STEP','33C 2H','33C 6H','33C 48H','36C STEP','36C 2H','36C 6H','36C 48H',...,'36C 6H','36C 48H','39C STEP','39C 2H','39C 6H','39C 48H','42C STEP','42C 2H','42C 6H','42C 48H'
4,YAL002W,CTCTATACTGACAGCACGCATGGCCGTACGCT,-0.111906986,-0.041303817,-0.04193078,-0.081481657,-0.068753292,-0.032223406,-0.030546997,-0.005880197,...,-0.026073517,-0.017706756,-0.034261957,-0.014238688,-0.031041245,-0.028411346,-0.801352653,-0.027469892,-0.017146153,-0.075210221


In [20]:
original_data.drop(columns=['Temperature Magnitude'], inplace=True)

In [22]:
original_data.drop(index=[0,1,2,3], inplace=True)

In [26]:
original_data['orf'] = original_data['Unnamed: 0'].astype(str)

In [27]:
# Eliminate all white spaces & capitalize
original_data['orf'] = clean_orf(original_data['orf'])

In [28]:
# Translate to ORFs 
original_data['orf'] = translate_sc(original_data['orf'], to='orf')

In [29]:
# Make sure everything translated ok
t = looks_like_orf(original_data['orf'])
print(original_data.loc[~t,])

Empty DataFrame
Columns: [Unnamed: 0, 30, 33, 33.1, 33.2, 36, 36.1, 36.2, 36.3, 39, 39.1, 39.2, 39.3, 42, 42.1, 42.2, 42.3, 30.1, 33.3, 33.4, 33.5, 36.4, 36.5, 36.6, 36.7, 39.4, 39.5, 39.6, 39.7, 42.4, 42.5, 42.6, 42.7, 30.2, 33.6, 33.7, 33.8, 36.8, 36.9, 36.10, 36.11, 39.8, 39.9, 39.10, 39.11, 42.8, 42.9, 42.10, 42.11, orf]
Index: []

[0 rows x 50 columns]


In [30]:
original_data.set_index('orf', inplace=True)

In [31]:
original_data.drop(columns=['Unnamed: 0'], inplace=True)

In [32]:
original_data.columns = datasets.index.values

In [33]:
original_data = original_data.apply(pd.to_numeric, axis=1)

In [34]:
original_data = original_data.groupby(original_data.index).mean()

In [35]:
original_data.shape

(4962, 48)

# Prepare the final dataset

In [36]:
data = original_data.copy()

In [37]:
dataset_ids = original_data.columns.values
datasets = datasets.reindex(index=dataset_ids)

In [38]:
lst = [datasets.index.values, ['value']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data.columns = idx

In [39]:
data.head()

dataset_id,21962,21959,21976,21980,21965,21982,21989,21985,21968,21991,...,21990,21987,21970,21993,21999,21996,21973,22002,22008,22005
data_type,value,value,value,value,value,value,value,value,value,value,...,value,value,value,value,value,value,value,value,value,value
orf,,,,,,,,,,,,,,,,,,,,,
YAL002W,-0.111907,-0.041304,-0.041931,-0.081482,-0.068753,-0.032223,-0.030547,-0.005880,-0.105876,-0.054339,...,-0.026074,-0.017707,-0.034262,-0.014239,-0.031041,-0.028411,-0.801353,-0.027470,-0.017146,-0.075210
YAL004W,0.047175,-0.049025,-0.019856,-0.044029,-0.023282,-0.021175,-0.038480,-0.034854,-0.150124,0.001560,...,-0.027285,-0.029912,-0.032301,-0.012680,-0.018477,-0.088925,-0.795017,-0.072851,-0.024553,-0.173336
YAL005C,-0.042938,-0.006418,-0.032269,-0.005857,-0.015218,-0.060341,0.018152,-0.033578,-0.039204,-0.019129,...,-0.018335,-0.029595,-0.011378,-0.028947,-0.005335,-0.007970,-0.784402,-0.096669,-0.026905,-0.028360
YAL007C,-0.006674,0.004075,-0.009083,-0.004575,0.010546,0.004288,-0.005632,0.001614,0.007129,-0.013179,...,-0.025784,-0.014553,0.003250,-0.008514,0.000873,-0.001985,-0.919837,0.003699,-0.001810,-0.199641
YAL008W,0.009356,-0.006416,-0.002276,0.005772,0.007862,0.005437,0.001569,0.008831,0.002523,0.002798,...,-0.015194,-0.009216,-0.003606,-0.002081,0.001009,0.001272,-0.922397,-0.003058,0.003621,-0.036458


## Subset to the genes currently in SGD

In [40]:
genes = pd.read_csv(path_to_genes, sep='\t', index_col='id')
genes = genes.reset_index().set_index('systematic_name')
gene_ids = genes.reindex(index=data.index.values)['id'].values
num_missing = np.sum(np.isnan(gene_ids))
print('ORFs missing from SGD: %d' % num_missing)

ORFs missing from SGD: 24


In [41]:
data['gene_id'] = gene_ids
data = data.loc[data['gene_id'].notnull()]
data['gene_id'] = data['gene_id'].astype(int)
data = data.reset_index().set_index(['gene_id','orf'])

data.head()

,dataset_id,21962,21959,21976,21980,21965,21982,21989,21985,21968,21991,...,21990,21987,21970,21993,21999,21996,21973,22002,22008,22005
,data_type,value,value,value,value,value,value,value,value,value,value,...,value,value,value,value,value,value,value,value,value,value
gene_id,orf,,,,,,,,,,,,,,,,,,,,,
2,YAL002W,-0.111907,-0.041304,-0.041931,-0.081482,-0.068753,-0.032223,-0.030547,-0.005880,-0.105876,-0.054339,...,-0.026074,-0.017707,-0.034262,-0.014239,-0.031041,-0.028411,-0.801353,-0.027470,-0.017146,-0.075210
1863,YAL004W,0.047175,-0.049025,-0.019856,-0.044029,-0.023282,-0.021175,-0.038480,-0.034854,-0.150124,0.001560,...,-0.027285,-0.029912,-0.032301,-0.012680,-0.018477,-0.088925,-0.795017,-0.072851,-0.024553,-0.173336
4,YAL005C,-0.042938,-0.006418,-0.032269,-0.005857,-0.015218,-0.060341,0.018152,-0.033578,-0.039204,-0.019129,...,-0.018335,-0.029595,-0.011378,-0.028947,-0.005335,-0.007970,-0.784402,-0.096669,-0.026905,-0.028360
5,YAL007C,-0.006674,0.004075,-0.009083,-0.004575,0.010546,0.004288,-0.005632,0.001614,0.007129,-0.013179,...,-0.025784,-0.014553,0.003250,-0.008514,0.000873,-0.001985,-0.919837,0.003699,-0.001810,-0.199641
6,YAL008W,0.009356,-0.006416,-0.002276,0.005772,0.007862,0.005437,0.001569,0.008831,0.002523,0.002798,...,-0.015194,-0.009216,-0.003606,-0.002081,0.001009,0.001272,-0.922397,-0.003058,0.003621,-0.036458


# Normalize

In [42]:
data_norm = normalize_phenotypic_scores(data, has_tested=True)

In [43]:
# Assign proper column names
lst = [datasets.index.values, ['valuez']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data_norm.columns = idx

In [44]:
data_norm[data.isnull()] = np.nan
data_all = data.join(data_norm)

data_all.head()

,dataset_id,21962,21959,21976,21980,21965,21982,21989,21985,21968,21991,...,21990,21987,21970,21993,21999,21996,21973,22002,22008,22005
,data_type,value,value,value,value,value,value,value,value,value,value,...,valuez,valuez,valuez,valuez,valuez,valuez,valuez,valuez,valuez,valuez
gene_id,orf,,,,,,,,,,,,,,,,,,,,,
2,YAL002W,-0.111907,-0.041304,-0.041931,-0.081482,-0.068753,-0.032223,-0.030547,-0.005880,-0.105876,-0.054339,...,-0.572223,-0.237039,-0.697130,-0.521732,-1.059506,-0.900926,0.053318,-0.832419,-0.560521,0.099712
1863,YAL004W,0.047175,-0.049025,-0.019856,-0.044029,-0.023282,-0.021175,-0.038480,-0.034854,-0.150124,0.001560,...,-0.625003,-0.630190,-0.654853,-0.419549,-0.594121,-2.796044,0.066917,-2.505183,-0.801836,-1.517441
4,YAL005C,-0.042938,-0.006418,-0.032269,-0.005857,-0.015218,-0.060341,0.018152,-0.033578,-0.039204,-0.019129,...,-0.235063,-0.619983,-0.203836,-1.485974,-0.107355,-0.260772,0.089703,-3.383130,-0.878458,0.871820
5,YAL007C,-0.006674,0.004075,-0.009083,-0.004575,0.010546,0.004288,-0.005632,0.001614,0.007129,-0.013179,...,-0.559627,-0.135435,0.111500,-0.146452,0.122607,-0.073324,-0.201010,0.316483,-0.060897,-1.950966
6,YAL008W,0.009356,-0.006416,-0.002276,0.005772,0.007862,0.005437,0.001569,0.008831,0.002523,0.002798,...,-0.098202,0.036481,-0.036281,0.275303,0.127652,0.028680,-0.206504,0.067435,0.116031,0.738368


# Print out

In [45]:
for f in ['value','valuez']:
    df = data_all.xs(f, level='data_type', axis=1).copy()
    df.columns = datasets['name'].values
    df = df.droplevel('gene_id', axis=0)
    df.to_csv(paper_name + '_' + f + '.txt', sep='\t')

# Save to DB

In [46]:
from IO.save_data_to_db3 import *

In [47]:
save_data_to_db(data_all, paper_pmid)

  0%|          | 0/48 [00:00<?, ?it/s]

Deleting all datasets for PMID 29889214...
Inserting the new data...


100%|██████████| 48/48 [04:28<00:00,  5.59s/it]

Updating the data_modified_on field...
